In [25]:
from aocd import get_data
data = get_data(day=16, year=2023)

In [3]:
import numpy as np

In [38]:
with open('.day16.txt', 'r') as f:
    data = f.read().strip()

In [39]:
board = np.array([ list(x) for x in data.split('\n') ])
energy = np.zeros(board.shape, dtype=int)
for r in board:
    print(''.join(r))

.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....


In [ ]:
ships = [ E((0,0)) ]

def Op:
    def __init__(self, pos):
        self.pos = pos
class N:
    pass

class E:
    def fly(self):
        pass

class S:
    pass

class W:
    pass

def fly(ship):
    